In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
# create spark configuration
conf = SparkConf()
conf.setMaster('local[2]')
conf.setAppName("TwitterStreamApp")
# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)
# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")


In [2]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
#             finally:
        hashtag_counts_df.show()
    except:
        e = sys.exc_info()[0]
        print(e)
    finally:
        try:
            hashtag_counts_df.show()
        except:
            pass
        

In [3]:
# read data from port 9009
dataStream = ssc.socketTextStream("localhost",9009)
dataStream.pprint()
# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))
# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))
# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
# start the streaming computation
ssc.start()
# wait for the streaming to finish
ssc.awaitTermination()

-------------------------------------------
Time: 2018-10-20 00:27:10
-------------------------------------------

----------- 2018-10-20 00:27:10 -----------
<class 'ValueError'>
-------------------------------------------
Time: 2018-10-20 00:27:12
-------------------------------------------
I made a bet that if the Astros lost last night I'd do the ol' handlebar. I did it 💀 https://t.co/1P2RJ3qr51
Spring Lake has tied it up off a fumble recovery; 17-17, 5:27 left in Q3. #SentPreps
I like this place! No wankers!!! @ Fayetteville, North Carolina https://t.co/1IJNPzQnfL
Watching #22July. #TeamNorway
@Olliesuperstar Shoot I forgot what I was gonna say🤦🏽‍♀️
Haunted house here I come😍🎃
Just posted a video @ Mi Rancho's Downtown, GA https://t.co/D8193Bh0Kh
Showered and oiled up. I got my fried Oreos and peach cobbler in bed wit me. 😍
Cedar Park brings the house and Chase Griffin takes advantage. His screen pass to Chux Nwabuko goes for a 33-yard t… https://t.co/pl5SgNemLl
@reedm @speakthelo

+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|     #WSSUHomecoming|            1|
|                #954|            1|
|         #MANICEATER|            1|
|#CarrolltonToLewi...|            1|
|            #traffic|            1|
|         #TrumpsDumb|            1|
|           #TheKing,|            1|
|    #TEMPLEvsMEMPHIS|            1|
|          #fallbreak|            1|
|   #zionnationalpark|            1|
+--------------------+-------------+

-------------------------------------------
Time: 2018-10-20 00:27:22
-------------------------------------------
DeAndra just said she eat gold fish like crack heads eat crack. Direct quote. 😑 I hate this bitch so bad. I quit being friends.
finally ordered a beto shirt am so excited
I just had a huge fight with my parents and I kind of want to go die
Or two*
subtle flexes &gt; doing the most, making it known that you're not used to having .
What an amazing day spent shooting 

KeyboardInterrupt: 

+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|     #WSSUHomecoming|            1|
|                #954|            1|
|         #MANICEATER|            1|
|#CarrolltonToLewi...|            1|
|            #traffic|            1|
|         #TrumpsDumb|            1|
|           #TheKing,|            1|
|    #TEMPLEvsMEMPHIS|            1|
|          #fallbreak|            1|
|   #zionnationalpark|            1|
+--------------------+-------------+

+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|     #WSSUHomecoming|            1|
|                #954|            1|
|         #MANICEATER|            1|
|#CarrolltonToLewi...|            1|
|            #traffic|            1|
|         #TrumpsDumb|            1|
|           #TheKing,|            1|
|    #TEMPLEvsMEMPHIS|            1|
|          #fallbreak|            1|
|   #zionnationalpark|            1|


+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|              #BLine|            3|
|              #5Line|            3|
|       #EBOBeardwash|            2|
|     #EBOBeardButter|            2|
|#TaconicStateParkway|            2|
|     #WSSUHomecoming|            1|
|                #954|            1|
|         #MANICEATER|            1|
|#CarrolltonToLewi...|            1|
|            #traffic|            1|
+--------------------+-------------+

-------------------------------------------
Time: 2018-10-20 00:27:36
-------------------------------------------
That's because they are soulless, bottomless, disgustingly cruel hypocrites.  🤷🤷
Join me and @yolandaharrisTV tomorrow night for a night of wrestling, as we support @RealJeremyRod and… https://t.co/nqRzAx4o06
🙃🙃my day is ruined
@rektredpill @RedCardRas At least it was safe sex
@ESUFOOTBALL @PittStGorillas  might get on this kid. Dude is a stud #3NOM
@DickieV @Brewe

...

-------------------------------------------
Time: 2018-10-20 00:28:54
-------------------------------------------
@taylurr_Leb Through the North Pacific Ocean and up the Gulf of Thailand. Don’t be late sha
🧡🥂 that’s what I’m talking about
After 1 quarter of play Tigers up 14-0 over Harrisburg
@SilhanaED There’s an erection joke in here somewhere..
Now playing : @Barbaric84 @adotalvarez585 " Grenade Rap " @JayRukkus @JFforeal    in rotation on… https://t.co/WMc3lIYg4V
@LiteCoinNews Hello, have an offer for you, could you please reply back with any of your contacts? Cheers
@LhyricLee They got dallas messed up !!!
Are you fnkidding me.
@rarrlow I can def agree with all of that. So you think lakers win a few games or a few series?
ain’t too many people that was there for me
...

----------- 2018-10-20 00:28:54 -----------
+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|       #EBOBeardwash|            8|
|     #EBOBeardButter|

+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|     #EBOBeardButter|            8|
|       #EBOBeardwash|            8|
|              #wisfb|            6|
|          #Halloween|            3|
|              #BLine|            3|
|               #PCAs|            3|
|#MyTwitterAnniver...|            3|
|          #WhyIWrite|            3|
|             #txhsfb|            3|
|              #5Line|            3|
+--------------------+-------------+

+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|     #EBOBeardButter|            8|
|       #EBOBeardwash|            8|
|              #wisfb|            6|
|          #Halloween|            3|
|              #BLine|            3|
|               #PCAs|            3|
|#MyTwitterAnniver...|            3|
|          #WhyIWrite|            3|
|             #txhsfb|            3|
|              #5Line|            3|


-------------------------------------------
Time: 2018-10-20 00:29:12
-------------------------------------------
The #kod #kodtour was awesome 2nd year going to see #jcole! Will see him again if he tours again! Next time come to… https://t.co/k3sUVh6Ngb
@errolbarnett @CBSNews You must be retarded.  Trying to interview someone while in process of boarding an airplane.… https://t.co/1LWgrNpglJ
I’m so glad Zo and I are talking again I love that girl ❤️
What I want right now is an entire basket of kittens that I get to keep.
Papi tried so hard today and got @katienolan at the end!! She was doing so well... and even said something in the b… https://t.co/A0hEb460JQ
@jammalama @mckenna188 Happy Birthday #wow
I genuinely do not know why anyone would support this atrocious party.
@quintonmarksii @U93 Soooo cool man! Make those dreams happen!
@AuriaStorm We have so many in the freezer I can bring you some Monday.
@kbergen28 True. But with the hype around VSOJ it’s still a bit surprising.
...

-

+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|     #EBOBeardButter|            8|
|       #EBOBeardwash|            8|
|              #wisfb|            6|
|          #WhyIWrite|            5|
|               #IDOL|            4|
|            #TheSong|            4|
|              #BLine|            3|
|#OtherScoobyDooMy...|            3|
|#MyTwitterAnniver...|            3|
|             #txhsfb|            3|
+--------------------+-------------+

+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|     #EBOBeardButter|            8|
|       #EBOBeardwash|            8|
|              #wisfb|            6|
|          #WhyIWrite|            5|
|               #IDOL|            4|
|            #TheSong|            4|
|              #BLine|            3|
|#OtherScoobyDooMy...|            3|
|#MyTwitterAnniver...|            3|
|             #txhsfb|            3|


+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|     #EBOBeardButter|            8|
|       #EBOBeardwash|            8|
|              #wisfb|            6|
|          #WhyIWrite|            5|
|               #IDOL|            4|
|            #TheSong|            4|
|              #BLine|            3|
|#OtherScoobyDooMy...|            3|
|#MyTwitterAnniver...|            3|
|             #txhsfb|            3|
+--------------------+-------------+

-------------------------------------------
Time: 2018-10-20 00:29:30
-------------------------------------------
Let’s just do it and be legends.
@PatriciaTully6 @kelleratlarge He outshines her in every way
Navy kills off the penalty back to even strength
It’s time for... you know
@Hall46Lavada Haha always
@Debtwielder Brought to you by the letter M 

#AllStarTrek
And a name change. Because I sort of don't like the name. (😂) At least shortened, perhaps to Retro Party. Geez.
D